# \[Old\] Baseline Model - MLP, PyTorch version

Doesn't work, if at all. Dependencies:
```
- numpy
- pandas
- torch (with CUDA support and torchvision)
- torchmetrics
- matplotlib
- seaborn
- sklearn
```

### Importing libraries and reading files

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']="1" # CUDA debug help
# Import statements
import numpy as np
import pandas as pd
import torch
import sys
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Check GPU and CPU count
print("================ GPU availability, CPU count and GPU status check ===============")
print(torch.cuda.get_device_name() if torch.cuda.is_available() else "No GPU")
print(os.cpu_count())
!nvidia-smi

================ GPU availability, CPU count and GPU status check ===============
GeForce RTX 2080 Ti
4
Thu Jan  6 19:24:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:05:00.0 Off |                  N/A |
| 18%   30C    P8     5W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------

In [3]:
# Set path (optional)
'''dataset_dir = '../Dataset'
sys.path.append(dataset_dir)'''
!pwd

/userhome/cs/u3556490/efyp/Baseline


In [4]:
# read Dataset as Pandas DataFrame
print("================ Reading dataset, null check ===============")
dataset = pd.read_pickle('../Dataset/baseline_dataset.gz')
pd.set_option("display.max.columns", None)
print(dataset.info())

================ Reading dataset, null check ===============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45293 entries, 0 to 45292
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MM             45293 non-null  int64  
 1   DD             45293 non-null  int64  
 2   HH             45293 non-null  int64  
 3   LOW_IMPACT     45293 non-null  bool   
 4   MID_IMPACT     45293 non-null  bool   
 5   BIG_IMPACT     45293 non-null  bool   
 6   DIRECT_STRIKE  45293 non-null  bool   
 7   00LAT          45293 non-null  float32
 8   00LON          45293 non-null  float32
 9   00WIND         45293 non-null  int32  
 10  06LAT          45293 non-null  float32
 11  06LON          45293 non-null  float32
 12  06WIND         45293 non-null  int32  
 13  12LAT          45293 non-null  float32
 14  12LON          45293 non-null  float32
 15  12WIND         45293 non-null  int32  
 16  18LAT          45293 non-null  fl

In [5]:
# san-check: any nulls?
print("Null checking:")
print(dataset[dataset.isnull().any(axis=1)])

Null checking:
Empty DataFrame
Columns: [MM, DD, HH, LOW_IMPACT, MID_IMPACT, BIG_IMPACT, DIRECT_STRIKE, 00LAT, 00LON, 00WIND, 06LAT, 06LON, 06WIND, 12LAT, 12LON, 12WIND, 18LAT, 18LON, 18WIND, 24LAT, 24LON, 24WIND]
Index: []


In [6]:
# dataset.describe()

,MM,DD,HH,00LAT,00LON,00WIND,06LAT,06LON,06WIND,12LAT,12LON,12WIND,18LAT,18LON,18WIND,24LAT,24LON,24WIND
count,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000,45293.000000
mean,8.283068,16.061025,8.918111,20.017653,132.009323,62.021063,19.506798,132.481476,62.087077,19.018631,132.967087,61.773122,18.551769,133.451675,61.072020,18.104473,133.910690,60.020864
std,2.310187,8.633509,6.698948,7.507561,19.457451,31.469743,7.243189,18.568924,31.423058,7.016464,17.948254,31.615497,6.824604,17.685791,31.978523,6.662520,17.910883,32.433757
min,1.000000,1.000000,0.000000,1.300000,-179.899994,10.000000,1.300000,-179.899994,10.000000,1.300000,-179.899994,10.000000,1.300000,-179.899994,10.000000,1.300000,-180.000000,10.000000
25%,7.000000,9.000000,0.000000,14.500000,121.800003,35.000000,14.200000,122.400002,35.000000,13.900000,123.000000,35.000000,13.600000,123.500000,35.000000,13.200000,124.000000,35.000000
50%,8.000000,16.000000,6.000000,19.200001,131.300003,55.000000,18.799999,131.600006,55.000000,18.400000,132.000000,55.000000,18.000000,132.399994,55.000000,17.600000,132.800003,55.000000
75%,10.000000,23.000000,12.000000,24.700001,142.399994,80.000000,24.100000,142.699997,80.000000,23.500000,143.000000,80.000000,22.900000,143.399994,80.000000,22.299999,143.899994,80.000000
max,12.000000,31.000000,23.000000,55.500000,180.000000,185.000000,54.500000,180.000000,185.000000,53.500000,180.000000,185.000000,52.000000,180.000000,185.000000,50.500000,180.000000,185.000000


### Prepare dataset for model to read

In [15]:
# constants
TRAIN_RATIO = 0.875
NUM_EPOCHS = 3
LR = 0.001
BATCH_SIZE = 256

In [7]:
# test-dev-train split
from sklearn.model_selection import train_test_split

def separateDataset(dataset, train_ratio):
    '''
    Takes in a dataset (pandas df) and a ratio value, returns a dictionary containing the separated dataset.
    Key "train" = train set, "dev" = dev set (size = train ratio * (sizeof input df - test set)), "test" = test set (size = train ratio * sizeof input df)
    '''
    train_dev_set, test_set = train_test_split(dataset, train_size=train_ratio)
    train_set, dev_set = train_test_split(train_dev_set, train_size=0.8)
    print("Training set size: {0}; Dev set size: {1}; Testing set size: {2}".format(len(train_set), len(dev_set), len(test_set)))
    return { "train": train_set, "dev": dev_set, "test": test_set }

In [23]:
from torch.utils.data import Dataset, DataLoader

# Dataset class to feed into the model
class BaselineDataset(Dataset):
    '''
    Custom dataset class to generate samples (X) and targets (Y) for the model.
    '''
    def __init__(self, dataframe):
        self.df = dataframe.drop(['LOW_IMPACT', 'MID_IMPACT', 'BIG_IMPACT', 'DIRECT_STRIKE'], axis=1)
        self.labels = np.asarray(dataframe.iloc[:,3:7]).astype(float)
  
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        rowToUse = torch.tensor(self.df.iloc[idx]).float()
        labelVector = torch.tensor(self.labels[idx]).float()
        return (rowToUse, labelVector)

In [24]:
splitDataset = separateDataset(dataset, TRAIN_RATIO)

train_set = BaselineDataset(splitDataset["train"])
dev_set = BaselineDataset(splitDataset["dev"])
test_set = BaselineDataset(splitDataset["test"])

train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
dev_dl = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_dl = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

Training set size: 31704; Dev set size: 7927; Testing set size: 5662


### Model definition

In [25]:
import torch.nn as nn

class BaselineMLP(nn.Module):
    '''
    The MLP model is defined here.
    '''
    def __init__(self):
        super(BaselineMLP, self).__init__()
        
        # input shape: (batch_size, 18)
        # output shape: (batch_size, 4)
        
        self.fc = nn.Sequential(
            nn.Linear(18, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 4), 
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.fc(x)
        return x

### Training Procedures

In [10]:
from sklearn.metrics import accuracy_score, f1_score
import torchmetrics

# calculate accuracies and such
def accuracy(output, target):
    '''
    Attempts to calculate accuracy and f1-score from the given outputs and targets (both on GPU).
    '''
    with torch.no_grad():
        #batch_size = output.size(0)
        
        target = target.cpu()
        preds = torch.round(output).cpu()
        acc = accuracy_score(target, preds)
        # acc = torchmetrics.functional.accuracy(preds, target.int(), num_classes=2, threshold=0.5, multiclass=True)        
        f1 = f1_score(target, preds, average=None, zero_division=0)
    
    return acc, f1

In [11]:
# train for one epoch

def train_model_for_one_epoch(model, train_dl, criterion, optimizer):
    '''The procedure to train the model for one epoch using the training set dataloader, loss function (criterion) and optimizer supplied'''
    
    model.train()
    
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = np.array([0.0, 0.0, 0.0, 0.0])
        
    for i, (inputs, targets) in enumerate(train_dl):
        inputs = inputs.cuda()
        targets = targets.cuda()
        
        # training steps
        optimizer.zero_grad()
        y = model(inputs)
        loss = criterion(y, targets)
        loss.backward()
        optimizer.step()

        # evaluate accuracy during training
        acc, f1 = accuracy(y, targets)
        
        # book-keeping: average this epoch
        running_loss += loss.item()
        running_acc += acc
        running_f1 += f1
    
    average_loss = running_loss/len(train_set)
    average_acc = running_acc/len(train_set)
    average_f1 = np.divide(running_f1, len(train_set))
    
    return average_acc, average_f1, average_loss

In [12]:
# evaluate on dev set

def evaluate(model, dev_dl, criterion):
    '''The procedure to evaluate the given model on the specified dev set (dataloader) using the supplied criterion (loss/objective)'''
    
    model.eval()
    
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = np.array([0.0, 0.0, 0.0, 0.0])
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(dev_dl):
            inputs = inputs.cuda()
            targets = targets.cuda()
            
            y = model(inputs)
            loss = criterion(y, targets)
            
            acc, f1 = accuracy(y, targets)
            
            running_loss += loss.item()
            running_acc += acc
            running_f1 += f1
            
    return running_acc/len(dev_set), np.divide(running_f1, len(dev_set)), running_loss/len(dev_set)

In [13]:
# main training loop
from torch.optim import Adam, SGD
import time

def main_training_loop(model, train_dl, dev_dl):
    '''Main training procedure, trains the model for some number of epochs, evaluating it at the end of each one'''
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=LR)
    #optimizer = SGD(model.parameters(), lr=LR)
    
    train_acc_history = []
    train_loss_history = []
    dev_acc_history = []
    dev_loss_history = []
    
    model.cuda()
        
    start_time = time.time()
    print("Main training loop is starting.")
    
    for epoch in range(NUM_EPOCHS):
        epoch_start = time.time()
        
        train_acc, train_f1, train_loss = train_model_for_one_epoch(model, train_dl, criterion, optimizer)
        train_acc_history.append(train_acc)
        train_loss_history.append(train_loss)
        
        dev_acc, dev_f1, dev_loss = evaluate(model, dev_dl, criterion)
        dev_acc_history.append(dev_acc)
        dev_loss_history.append(dev_loss)
        
        epoch_time = time.time() - epoch_start
        
        if epoch % 1 == 0:
            print("Progress: {0}/{1} epochs, train acc = {2:.5f}, dev acc = {3:.5f}, train loss = {4:.5f}, epoch time = {5:.2f}s".format(epoch+1, NUM_EPOCHS, train_acc, dev_acc, train_loss, epoch_time))

    training_time = time.time() - start_time
    print("Training has completed. Time elapsed = {0}s".format(training_time))
    
    training_history = { 'loss': train_loss_history, 'acc': train_acc_history }
    validate_history = { 'loss': dev_loss_history, 'acc': dev_acc_history }
    
    return training_history, validate_history

In [14]:
# testing on test set
from sklearn.metrics import precision_score, recall_score

def test_model(model, test_dl):
    '''Tests the model on the test set via the dataloader'''
    criterion = nn.BCELoss()
    
    model.eval()
    
    running_loss = 0.0
    running_acc = 0.0
    running_f1 = np.array([0.0, 0.0, 0.0, 0.0])
    running_precision = np.array([0.0, 0.0, 0.0, 0.0])
    running_recall = np.array([0.0, 0.0, 0.0, 0.0])
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(test_dl):
            inputs = inputs.cuda()
            targets = targets.cuda()
            
            y = model(inputs)
            loss = criterion(y, targets)
            
            acc, f1 = accuracy(y, targets)
            
            running_loss += loss.item()
            running_acc += acc
            running_f1 += f1
            
            y = torch.round(y).cpu()
            targets = targets.cpu()
            running_precision += precision_score(targets, y, average=None, zero_division=0)
            running_recall += recall_score(targets, y, average=None, zero_division=0)            
            
    final_avg_acc = running_acc/len(test_set)
    final_avg_loss = running_loss/len(test_set)
    final_avg_f1 = np.divide(running_f1, len(test_set))
    final_avg_precision = np.divide(running_precision, len(test_set))
    final_avg_recall = np.divide(running_recall, len(test_set))
    
    return {'acc': final_avg_acc, 'f1': final_avg_f1, 'loss': final_avg_loss, 'precision': final_avg_precision, 'recall': final_avg_recall}

In [26]:
print("================ Main training and evaluation procedure ===============")
model = BaselineMLP()
training_history, validate_history = main_training_loop(model, train_dl, dev_dl)
results = test_model(model, test_dl)
print(results)

================ Main training and evaluation procedure ===============
Main training loop is starting.
Progress: 1/3 epochs, train acc = 0.00338, dev acc = 0.00334, train loss = 0.00079, epoch time = 12.50s
Progress: 2/3 epochs, train acc = 0.00344, dev acc = 0.00334, train loss = 0.00068, epoch time = 12.26s
Progress: 3/3 epochs, train acc = 0.00344, dev acc = 0.00334, train loss = 0.00065, epoch time = 11.30s
Training has completed. Time elapsed = 36.05869913101196s
{'acc': 0.0034315944012716353, 'f1': array([0., 0., 0., 0.]), 'loss': 0.0006630018337671515, 'precision': array([0., 0., 0., 0.]), 'recall': array([0., 0., 0., 0.])}


#### Saving training results to file

In [15]:
# save model as checkpoint
print("================ Saving results to file ===============")

from datetime import datetime
torch.save({
    'model_state_dict': model.state_dict(),
}, "./models/baseline_model_{0}.pt".format(str(datetime.now().strftime("%Y-%m-%d %H:%M"))))

# save histories to file
df = pd.DataFrame.from_dict(training_history)
df.to_csv("./models/baseline_model_{0}_training.csv".format(str(datetime.now().strftime("%Y-%m-%d %H:%M"))))
df = pd.DataFrame.from_dict(validate_history)
df.to_csv("./models/baseline_model_{0}_validate.csv".format(str(datetime.now().strftime("%Y-%m-%d %H:%M"))))

# save scores to file
array_like_scores = { key: value for key, value in results.items() if key in ["f1", "precision", "recall"] }
df = pd.DataFrame.from_dict(array_like_scores)
df.to_csv("./models/baseline_model_{0}_array_scores.csv".format(str(datetime.now().strftime("%Y-%m-%d %H:%M"))))
general_scores = { key: [value] for key, value in results.items() if key in ["acc", "loss"] }
df = pd.DataFrame.from_dict(general_scores)
df.to_csv("./models/baseline_model_{0}_general_scores.csv".format(str(datetime.now().strftime("%Y-%m-%d %H:%M"))))

================ Saving results to file ===============
